In [1]:
# Copyright (C) 2016 Delaters
# This file is part of PARTERRA_GEE.
#
# PARTERRA_GEE is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# PARTERRA_GEE is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
#  You should have received a copy of the GNU General Public License
#  along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
import ogr
import os
import shapely, shapely.wkt

In [2]:
path='daressalaam-latest.db'
path=r'd:\downloads\daressalaam-20160619.db'
ds=ogr.Open(path)
for i in range(ds.GetLayerCount()):
    print(str(i) + ': ' + str(ds.GetLayerByIndex(i).GetName()))

0: points
1: lines
2: multilinestrings
3: multipolygons
4: other_relations


In [3]:
def convert(input_path, output_path, output_format, replace_fields, start_feature_index=0, layer_index=0):
    filename = os.path.splitext(os.path.basename(input_path))[0]

    dst_ds = ogr.GetDriverByName(output_format).CreateDataSource(output_path)
    dst_lyr = dst_ds.CreateLayer(filename)

    # create fields using OGR
    src_ds = ogr.Open(input_path)
    src_lyr = src_ds.GetLayerByIndex(layer_index)
    f = src_lyr.GetFeature(start_feature_index)
    
    field_type_map = {
        int: ogr.OFTInteger,
        float: ogr.OFTReal,
        str: ogr.OFTString
    }
    
    for i in range(f.GetFieldCount()):
        fd = f.GetFieldDefnRef(i)
        
        if fd.GetName() in replace_fields:
            fd = ogr.FieldDefn(fd.GetName(), field_type_map[replace_fields[fd.GetName()]])
            
            fd_isempty = ogr.FieldDefn(fd.GetName() + '_isempty', ogr.OFTInteger)
            dst_lyr.CreateField(fd_isempty)

        dst_lyr.CreateField(fd)
        
    # for CSV: create field for KML geometry
    # dst_lyr.CreateField(ogr.FieldDefn('kml', ogr.OFTString))

    bad = 0
    
    for feat in src_lyr:
    # for feat in [src_lyr.GetFeature(1)]: # DEBUG
        try:
            geom = shapely.wkt.loads(feat.GetGeometryRef().ExportToWkt())
        except Exception as e: 
            # print('Error({0}), skipping geometry.'.format(e))
            continue

        f = ogr.Feature(dst_lyr.GetLayerDefn())

        # set field values
        for i in range(feat.GetFieldCount()):
            fieldDef = feat.GetFieldDefnRef(i)
            fieldName = fieldDef.GetName()
            field = feat.GetField(fieldName)
            
            empty = 0
            
            if field:
                v = field
                
                if fieldName in replace_fields:
                    try:
                        v1 = replace_fields[fieldName](v) # parse
                        
                        if str(v) != str(v1):
                            print('{0} ({1}) -> {2} ({3})'.format(v, type(v), v1, type(v1)))
                        v = v1
                    except ValueError:
                        bad += 1
                        print('ValueError: error parsing value: {0} as {1}'.format(v, replace_fields[fieldName]))
                        
                        empty = 1
                    
                f.SetField(fieldName, v)
            else:
                empty = 1
                
                if fieldName in replace_fields:
                    f.SetField(fieldName, -1)
                else:
                    f.SetField(fieldName, '')
                
            if fieldName in replace_fields:
                f.SetField(fieldName + '_isempty', empty)

        # set geometry    
        f.SetGeometry(ogr.CreateGeometryFromWkt(geom.to_wkt()))
        # f.SetField('kml', ogr.CreateGeometryFromWkt(geom.to_wkt()).ExportToKML())
        
        # create feature
        dst_lyr.CreateFeature(f)
        
        f.Destroy() 
        
    print('Number of bad fields: {0}'.format(bad))
    
    src_ds.Destroy()

    dst_ds.Destroy()


In [4]:
replace_fields = {
    'building_levels': int,
    'height': float,
    'building': str
}

convert(path, 'daressalaam-20160619-polys.kml', 'kml', replace_fields, 1, 3)

ValueError: error parsing value: `1 as <type 'int'>
ValueError: error parsing value: A as <type 'int'>
ValueError: error parsing value: A as <type 'int'>
ValueError: error parsing value: A as <type 'int'>
ValueError: error parsing value: I as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
ValueError: error parsing value: Level 1 as <type 'int'>
Val

In [5]:
replace_fields = {
    'layer': int,
    'depth': float,
    'width': float,
}

convert(path, 'daressalaam-20160619-lines.kml', 'KML', replace_fields, 1, 1)

15 (<type 'str'>) -> 15.0 (<type 'float'>)
10 (<type 'str'>) -> 10.0 (<type 'float'>)
5 (<type 'str'>) -> 5.0 (<type 'float'>)
10 (<type 'str'>) -> 10.0 (<type 'float'>)
10 (<type 'str'>) -> 10.0 (<type 'float'>)
6 (<type 'str'>) -> 6.0 (<type 'float'>)
8 (<type 'str'>) -> 8.0 (<type 'float'>)
20 (<type 'str'>) -> 20.0 (<type 'float'>)
10 (<type 'str'>) -> 10.0 (<type 'float'>)
3 (<type 'str'>) -> 3.0 (<type 'float'>)
10 (<type 'str'>) -> 10.0 (<type 'float'>)
5 (<type 'str'>) -> 5.0 (<type 'float'>)
4 (<type 'str'>) -> 4.0 (<type 'float'>)
6 (<type 'str'>) -> 6.0 (<type 'float'>)
5 (<type 'str'>) -> 5.0 (<type 'float'>)
6 (<type 'str'>) -> 6.0 (<type 'float'>)
4 (<type 'str'>) -> 4.0 (<type 'float'>)
4 (<type 'str'>) -> 4.0 (<type 'float'>)
10 (<type 'str'>) -> 10.0 (<type 'float'>)
10 (<type 'str'>) -> 10.0 (<type 'float'>)
10 (<type 'str'>) -> 10.0 (<type 'float'>)
1 (<type 'str'>) -> 1.0 (<type 'float'>)
8 (<type 'str'>) -> 8.0 (<type 'float'>)
6 (<type 'str'>) -> 6.0 (<type 'float

In [6]:
for i in range(ogr.GetDriverCount()):
    print(ogr.GetDriver(i).GetName())

PCIDSK
JP2OpenJPEG
PDF
ESRI Shapefile
MapInfo File
UK .NTF
OGR_SDTS
S57
DGN
OGR_VRT
REC
Memory
BNA
CSV
NAS
GML
GPX
LIBKML
KML
GeoJSON
Interlis 1
Interlis 2
OGR_GMT
GPKG
SQLite
ODBC
WAsP
PGeo
MSSQLSpatial
PostgreSQL
MySQL
OpenFileGDB
XPlane
DXF
Geoconcept
GeoRSS
GPSTrackMaker
VFK
PGDUMP
OSM
GPSBabel
SUA
OpenAir
OGR_PDS
WFS
HTF
AeronavFAA
Geomedia
EDIGEO
GFT
GME
SVG
CouchDB
Cloudant
Idrisi
ARCGEN
SEGUKOOA
SEGY
ODS
XLSX
ElasticSearch
Walk
CartoDB
SXF
Selafin
JML
PLSCENES
CSW
TIGER
AVCBin
AVCE00
HTTP
